In [19]:
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine
import time,gc,os,re,sys,re,jieba,os,math
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [20]:
def check_cols(df):
    cols=df.columns.tolist()
    leng=int(len(cols)/5)+1
    for i in range(leng):
        tmp=df.iloc[:5,i*6:(i+1)*6]
        print(tmp.head())
        print('-'*30)

def check_miss_ratio(df):
    missdf=pd.isnull(df).sum().reset_index(name='miss_cnt')
    missdf.columns=['col','miss_cnt']
    missdf['miss_ratio']=missdf['miss_cnt']/df.shape[0]
    print(missdf.head(20))
    return missdf

In [21]:
# great_dic={'乐玩美研':'BCL','欧莱雅':'巴黎欧莱雅',"L'OREAL":"L'OREALPARIS","自然乐园":"自然共和国","乔治阿玛尼":"阿玛尼",
#            "GIORGIOARMANI":"ARMANI","AEKYUNG":"AEKYUNGAGE'S","伊诗露":"伊特露"}

In [22]:
def auto_mark_1(s,k):
    leng=len(s)
    lst=[]
    
    summ=0
    for i in range(leng):
        word=s[i]
        if word!=k[0]:
            lst.append('O')
            summ+=1
        elif word==k[0]:
            lst.append('BRD-B')
            for j in range(i+1,leng):
                word1=s[j]
                if word1 in k:
                    lst.append('BRD-I')
                    summ+=1
                else:
                    break
    return s


def auto_mark_2(s,k):
    klist=get_key_brd(k)

    s_leng=len(s)
    
    final_s_2=[]
    if k in s:
        s=s.replace(k,'#BRD#')
        lst=s.split('#BRD#')
        lst1=lst[0]
        if lst1=='':
            final_s_2.extend(klist)
            for i in range(1,len(lst)):
                if lst[i]:
                    new_list=[x for x in lst[i]]
                    final_s_2.extend('O'*len(new_list))
                    if i!=len(lst)-1:
                        final_s_2.extend(klist)
                    else:
                        continue

        else:
            for i in range(len(lst)):
                if lst[i]:
                    new_list=[x for x in lst[i]]
                    final_s_2.extend('O'*len(new_list))
                    if i!=len(lst)-1:
                        final_s_2.extend(klist)
                    else:
                        continue
    return ','.join(final_s_2)


def auto_mark_3(s,k):

    klist=get_key_brd(k)
    
    lst=[]
    if s==k:
        lst.extend(klist)
    else:
        ss=s.split(k)
        for i in range(len(ss)):
            if ss[i]=='':
                if i!=len(ss)-1:
                     lst.extend(klist)
            else:
                lst.extend(['O' for x in ss[i]])
                if i!=len(ss)-1:
                    lst.extend(klist)
    leng=len(lst)
    return ','.join(lst),leng

In [23]:
def get_key_brd(k):
    if type(k)=='float':
        k=str(int(k))
    leng=len(k)
    lst=['BRD-B']
    for i in range(leng-1):
        lst.append('BRD-I')
    return lst

In [24]:
def get_key_brd_bieo(k):
    if type(k)=='float':
        k=str(int(k))
        
    leng=len(k)
    lst=['BRD-B']
    
    if leng<=2:
        for i in range(leng-1):
            lst.append('BRD-E')
    else:
        for i in range(leng-1-1):
            lst.append('BRD-M')
        lst.append('BRD-E')
    return lst

In [25]:
def auto_mark_33(s,k):
    if type(k)=='float':
        k=str(int(k))
        
    klist=get_key_brd(k)
    
    lst=[]
    if s==k:
        lst.extend(klist)
    else:
        ss=s.split(k)
        for i in range(len(ss)):
            if ss[i]=='':
                if i!=len(ss)-1:
                     lst.extend(klist)
            else:
                lst.extend(['O' for x in ss[i]])
                if i!=len(ss)-1:
                    lst.extend(klist)
    return ','.join(lst)

In [26]:
def auto_mark_bieo(s,k):
    if type(k)=='float':
        k=str(int(k))
    klist=get_key_brd_bieo(k)

    lst=[]
    if s==k:
        lst.extend(klist)
    else:
        ss=s.split(k)
        for i in range(len(ss)):
            if ss[i]=='':
                if i!=len(ss)-1:
                    lst.extend(klist)
            else:
                lst.extend(['O' for x in ss[i]])
                if i!=len(ss)-1:
                    lst.extend(klist)
    return ','.join(lst)

In [27]:
df=pd.read_csv('data_1.csv')
print(df.shape)

(4519931, 12)


In [28]:
df.loc[df['brand_dig'].notnull(),'brand_dig']=df[df['brand_dig'].notnull()]['brand_dig'].apply(lambda x:str(int(x)))

In [29]:
def further_clean(v):
    v=str(v).replace(' ','')
    v=v.upper()
    pattern='[^0-9A-Z\u4e00-\u9fa5]'
    v=re.sub(pattern,'',v)
    return v

In [30]:
str_cols=['brand', 'brand_cn', 'brand_cn_fill', 'brand_dig', 'brand_en', 'brand_en_fill', 'company_cn', 'company_en', 'desc']

for col in str_cols:
    df.loc[(df[col].notnull())&(df[col]!=''),col]=df[(df[col].notnull())&(df[col]!='')][col].apply(lambda x:further_clean(x))
    gc.collect()

In [31]:
new_brand_col_dict={'赫妍':'赫拉', '悦诗风吟':'风吟', '乔治阿玛尼':'阿玛尼', '伊蒂之屋':'爱丽小屋', '纪梵希':'紀梵希',
                    '珂莱欧':'可莱欧', '魅可':'MAC', '玛丽黛佳':'玛丽佳人', '资生堂':'資生堂',
                    '倩碧':'倩倩', '芭比波朗':'波比布朗', '兰蔻':'兰KOU', '菲诗小铺':'菲斯小铺',
                    "SWEETCOLOR":"SWEETCOLO",
                    "ILLAMASQUA":"ILLAMASUQA",
                    "姬芮":"ZA", "HOLIKAHOLIKA":"HOLIKA", "三熹玉":"3CE", "雪詩緹":"雪诗缇", "THREE":"777",
                    "曼秀雷敦":"曼秀雷", "SMITHCHU":"禇铁匠","DRECODE":"DRECDE"}

In [32]:
df.loc[df['brand_cn'].notnull(),'brand_cn_simple']=df[df['brand_cn'].notnull()]['brand_cn'].apply(lambda x:new_brand_col_dict[x] if x in new_brand_col_dict else np.nan)
df.loc[(df['brand_cn_simple'].isnull())&(df['brand_en'].notnull()),'brand_cn_simple']=df[(df['brand_cn_simple'].isnull())&(df['brand_en'].notnull())]['brand_en'].apply(lambda x:new_brand_col_dict[x] if x in new_brand_col_dict else np.nan)

In [33]:
for col in str_cols:
    df.loc[df[col]=='',col]=np.nan
    df.loc[df[col]=='nan',col]=np.nan
    gc.collect()

In [34]:
dff=df[df['desc'].notnull()]
print(dff.shape)

(4519472, 13)


In [35]:
test=dff[(dff['brand_cn'].isnull())&(dff['brand_cn_fill'].isnull())&(dff['brand_dig'].isnull())&(dff['brand_en'].isnull())&(dff['brand_en_fill'].isnull())&(dff['company_cn'].isnull())&(dff['company_en'].isnull())&(dff['brand_cn_simple'].isnull())]
print(test.shape)

(299402, 13)


In [36]:
train=dff[~dff['id'].isin(set(test['id']))]
print(train.shape)

(4220070, 13)


In [37]:
print(len(train)/len(dff))

0.9337528808674995


In [38]:
train=train.reset_index(drop=True)
train['leng']=train['desc'].apply(len)
gc.collect()

7

In [39]:
test=test.reset_index(drop=True)
test['leng']=test['desc'].apply(len)
gc.collect()

7

In [40]:
# for col in ['brand_cn', 'brand_cn_fill', 'brand_dig', 'brand_en', 'brand_en_fill', 'company_cn', 'company_en', 'brand_cn_simple']:
#     print(col)
#     train[col]=train[col].astype(str)

In [41]:
# check1=train[train['brand_cn']=='']
# print(check1.shape)
# check2=train[train['brand_cn'].isnull()]
# print(check2.shape)
# check3=train[train['brand_cn']=='nan']
# print(check3.shape)
# gc.collect()

In [42]:
train['brand_cn']=train['brand_cn'].astype(str)

In [43]:
def check_col(tmp,col):
    check1=tmp[tmp[col].isnull()]
    print(check1.shape)
    check2=tmp[tmp[col]=='']
    print(check2.shape)
    check3=tmp[tmp[col]=='nan']
    print(check3.shape)

In [44]:
check_col(train,'brand_cn')

(0, 14)
(0, 14)
(991616, 14)


In [45]:
print(pd.isnull(train[['id','brand_cn']]).sum())

id          0
brand_cn    0
dtype: int64


In [46]:
train['brand_cn_ner']=train[['desc','brand_cn']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [47]:
train['brand_cn_fill']=train['brand_cn_fill'].astype(str)

In [48]:
check_col(train,'brand_cn_fill')

(0, 15)
(0, 15)
(1410252, 15)


In [49]:
print(pd.isnull(train[['id','brand_cn_fill']]).sum())

id               0
brand_cn_fill    0
dtype: int64


In [50]:
train['brand_cn_fill_ner']=train[['desc','brand_cn_fill']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [51]:
train['brand_en']=train['brand_en'].astype(str)
train['brand_en_ner']=train[['desc','brand_en']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [52]:
train['brand_en_fill']=train['brand_en_fill'].astype(str)
train['brand_en_fill_ner']=train[['desc','brand_en_fill']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)
gc.collect()

28

In [53]:
train['company_cn']=train['company_cn'].astype(str)
train['company_cn_ner']=train[['desc','company_cn']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [54]:
train['company_en']=train['company_en'].astype(str)
train['company_en_ner']=train[['desc','company_en']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [55]:
train['brand_dig']=train['brand_dig'].astype(str)
train['brand_dig_ner']=train[['desc','brand_dig']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [56]:
train['brand_cn_simple']=train['brand_cn_simple'].astype(str)
train['brand_cn_simple_ner']=train[['desc','brand_cn_simple']].apply(lambda row:auto_mark_bieo(row[0],row[1]),raw=True,axis=1)

In [57]:
gc.collect()

28

In [58]:
def get_final_ner(v1,v2,v3,v4,v5,v6,v7,v8,leng):
    v1=list(v1.split(','))
    v2=list(v2.split(','))
    v3=list(v3.split(','))
    v4=list(v4.split(','))
    v5=list(v5.split(','))
    v6=list(v6.split(','))
    v7=list(v7.split(','))
    v8=list(v8.split(','))
    
    if len(v1)==len(v2)==len(v3)==leng:
        new_lst=[]
        for i in range(leng):
            v11=v1[i]
            v22=v2[i]
            v33=v3[i]
            v44=v4[i]
            v55=v5[i]
            v66=v6[i]
            v77=v7[i]
            v88=v8[i]
            if 'BRD-I' in [v11,v22,v33,v44,v55,v66,v77,v88]: #注意里面有顺序关系，比如悦诗风吟和风吟
                new_lst.append('BRD-I')
            elif 'BRD-B' in [v11,v22,v33,v44,v55,v66,v77,v88]:
                new_lst.append('BRD-B')
            else:
                new_lst.append('O')
        return ','.join(new_lst)
    else:
        return 'wrong'

In [59]:
def get_final_ner_bieo(v1,v2,v3,v4,v5,v6,v7,v8,leng):
    v1=list(v1.split(','))
    v2=list(v2.split(','))
    v3=list(v3.split(','))
    v4=list(v4.split(','))
    v5=list(v5.split(','))
    v6=list(v6.split(','))
    v7=list(v7.split(','))
    v8=list(v8.split(','))
    
    if len(v1)==len(v2)==len(v3)==leng:
        new_lst=[]
        for i in range(leng):
            v11=v1[i]
            v22=v2[i]
            v33=v3[i]
            v44=v4[i]
            v55=v5[i]
            v66=v6[i]
            v77=v7[i]
            v88=v8[i]
            if 'BRD-M' in [v11,v22,v33,v44,v55,v66,v77,v88]: #注意里面有顺序关系，比如悦诗风吟和风吟
                new_lst.append('BRD-M')
            elif 'BRD-E' in [v11,v22,v33,v44,v55,v66,v77,v88]:
                new_lst.append('BRD-E')
            elif 'BRD-B' in [v11,v22,v33,v44,v55,v66,v77,v88]:
                new_lst.append('BRD-B')
            else:
                new_lst.append('O')
        return ','.join(new_lst)
    else:
        return 'wrong'

In [60]:
s='悦诗风之吟最新优惠风之吟'
k1='悦诗风之吟'
v1=auto_mark_bieo(s,k1)
# print(v1)
k2='悦诗'
v2=auto_mark_bieo(s,k2)
# print(v2)
k3='风之吟'
v3=auto_mark_bieo(s,k3)
# print(v3)
v4=v1
v5=v1
v6=v2
v7=v3
v8=v2
leng=len(s)
f=get_final_ner_bieo(v1,v2,v3,v4,v5,v6,v7,v8,leng)
print(f)

BRD-B,BRD-M,BRD-M,BRD-M,BRD-E,O,O,O,O,BRD-B,BRD-M,BRD-E


In [61]:
train['final_ner']=list(map(lambda a,b,c,d,e,f,g,h,leng:get_final_ner_bieo(a,b,c,d,e,f,g,h,leng),train['brand_cn_ner'],train['brand_en_ner'],train['company_cn_ner'],train['company_en_ner'],train['brand_cn_simple_ner'],train['brand_cn_fill_ner'],train['brand_en_fill_ner'],train['brand_dig_ner'],train['leng']))
gc.collect()

7

In [62]:
check=train[train['final_ner']=='wrong']
print(check.shape)
if len(check)>0:
    check.to_csv('ner_wrong_check.csv',index=False,encoding='utf-8-sig')
gc.collect()

(0, 23)


0

In [ ]:
# name='bio'
# name='bieo'

In [63]:
train_df=train[train['final_ner']!='wrong']
print(len(train_df)/len(train))
train_df.to_csv('train_ner_%s.csv'%name,index=False,encoding='utf-8-sig')
gc.collect()

1.0


0

In [64]:
test_df=test.copy()
test_df.to_csv('test_ner_%s.csv'%name,index=False,encoding='utf-8-sig')
gc.collect()

0

In [65]:
gc.collect()

0

### <font color=red>检查train_ner

#### <font color=blue>(1)检查整体

In [67]:
print('开始检查整体')
for i in range(1200,1300,1):
    print(train_df['desc'][i])
    print(train_df['final_ner'][i])
    print(train_df['brand_cn'][i])
    print(train_df['brand_en'][i])
    print(train_df['brand'][i])
    print(train_df['brand_dig'][i])
    print('-'*40)

开始检查整体
现货APIEU温和眼唇卸妆液美妆节目一位产品100ML
O,O,BRD-B,BRD-M,BRD-M,BRD-M,BRD-E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
APIEU
APIEU
2050
2050
----------------------------------------
两只包邮现货COLOURPOP人鱼姬唇釉唇彩WOLFIEFLITTERKWEEN
O,O,O,O,O,O,BRD-B,BRD-M,BRD-M,BRD-M,BRD-M,BRD-M,BRD-M,BRD-M,BRD-E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
COLOURPOP
COLOURPOP
919
919
----------------------------------------
良品男装17年新款推荐爱美女神福里精灵气垫BB霜粉底女式
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------------
17年新款推荐爱美女神福里精灵气垫BB霜粉底女式
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------------
17年新款推荐爱美女神福里精灵气垫BB霜粉底女式
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------------
现货各品牌中小样集合6月已更新
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
919
919
----------------------------------------
612妖精美妈玫瑰卸妆水
O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
2050
2050
----------------------------------------
1

#### <font color=blue>(2)检查没有实体的数据

In [68]:
#真正需要检查的是,都是O的这样的情况,看看里面确实是没有关键字还是因为品牌的名字多写了一个字母或者少写了一个字母
def get_all_zero(v):
    v=v.split(',')
    s=0
    for w in v:
        if w=='O':
            s+=1
    if s==len(v):
        return 'all_zero'
    else:
        return 'have_key'

In [69]:
train_df['mark']=train_df['final_ner'].apply(get_all_zero)

In [70]:
check=train_df[train_df['mark']=='all_zero'].reset_index(drop=True)
print(check.shape)

(1189550, 24)


In [71]:
print('开始检查都是O的数据')
for i in range(100,200):
    print(check['desc'][i])
    print(check['final_ner'][i])
    print(check['brand_cn'][i])
    print(check['brand_en'][i])
    print('-'*40)
gc.collect()

开始检查都是O的数据
便携式伸缩唇刷热卖彩妆化妆工具适用唇彩唇膏唇蜜
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
天然环保竹子柄4支化妆刷套装腮红刷蜜粉刷眼影刷遮瑕刷包邮
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
正品包邮化妆师初学者10支经典化妆刷套刷彩妆工具全套化妆扫
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
指甲油按斤称厂家库存外贸尾货STOCK义乌小商品地摊2元店只批发
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
新款外贸热销款9支动物毛化妆刷套装便携款
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
大卷巾美容院必备全棉无纺布一次性洗面巾洁面巾美容巾化妆棉巾
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
魔幻彩色环保指甲油无毒无味孕妇儿童可用持久光泽易吸收美甲油
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
----------------------------------------
专业蘑菇刷刷桶便携式收纳包小刷桶
O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
nan
nan
------------

0